In [ ]:
import numpy as np
import time


# --- Definição dos Estados e Sinais (Baseado em um Dot Product de 4 elementos) ---


# Estados da FSM (10 estados, 4 bits de estado)
STATE_IDLE = 0
STATE_LOAD_SETUP = 1
STATE_LOAD_VECTORS = 2
STATE_ACCUMULATOR_RESET = 3
STATE_EXECUTE_0 = 4   # Multiplica A0*B0
STATE_EXECUTE_1 = 5   # Multiplica A1*B1
STATE_EXECUTE_2 = 6   # Multiplica A2*B2
STATE_EXECUTE_3 = 7   # Multiplica A3*B3
STATE_WRITE_BACK = 8
STATE_DONE = 9


class FSM_Controller:
    """
    Simula a Máquina de Estados Finitos (FSM) para o Acelerador Vetorial.
    Verifica transições e geração de sinais de controle (outputs).
    """
    def __init__(self):
        self.state = STATE_IDLE
        self.output_signals = {
            'addr_w_ctrl': np.int8(0),     # Endereço de escrita na BRAM
            'we_ctrl': np.int8(0),         # Write Enable da BRAM
            'acc_rst': np.int8(0),         # Reset do Acumulador
            'mux_4_1_sel': np.int8(0),     # Seleção do MUX 4:1 (0 a 3)
            'mux_2_1_res_sel': np.int8(0), # Seleção do MUX 2:1 (0=ALU, 1=ACC)
            'done_out': np.int8(0)         # Sinal de conclusão
        }


    def reset(self):
        """ Simula o sinal de reset assíncrono. """
        self.state = STATE_IDLE
        # Zera todos os sinais de saída (segurança)
        for key in self.output_signals:
            self.output_signals[key] = np.int8(0)
        return self.state


    def update_state(self, start_in):
        """ Simula a transição de estado no flanco de subida do clock. """
       
        # 1. Lógica Combinacional (Próximo Estado e Sinais de Saída)
        next_state = self.state
       
        # Estado de espera
        if self.state == STATE_IDLE:
            self._set_outputs(we=0, acc_rst=0, mux_res_sel=0, done=0)
            if start_in == 1:
                next_state = STATE_LOAD_SETUP
       
        # Leitura de Parâmetros Iniciais (Endereço, etc.)
        elif self.state == STATE_LOAD_SETUP:
            self._set_outputs(we=0, acc_rst=0, mux_res_sel=0, done=0)
            # Simula a leitura dos registradores de controle
            next_state = STATE_LOAD_VECTORS
           
        # Carregamento dos Vetores (A e B)
        elif self.state == STATE_LOAD_VECTORS:
            self._set_outputs(we=0, acc_rst=0, mux_res_sel=0, done=0)
            next_state = STATE_ACCUMULATOR_RESET # Próximo: Preparar Acumulador
           
        # Reset do Acumulador
        elif self.state == STATE_ACCUMULATOR_RESET:
            self._set_outputs(we=0, acc_rst=1, mux_res_sel=0, done=0) # ACUMULATOR_RESET = 1
            next_state = STATE_EXECUTE_0
           
        # Loop de Execução (Multiplicação e Acumulação: 4 ciclos)
        elif STATE_EXECUTE_0 <= self.state <= STATE_EXECUTE_3:
            # Seleção do MUX 4:1 (0 a 3) é igual ao offset do estado (state - 4)
            mux_sel = self.state - STATE_EXECUTE_0
            self._set_outputs(we=0, acc_rst=0, mux_sel=mux_sel, mux_res_sel=0, done=0)


            if self.state < STATE_EXECUTE_3:
                next_state = self.state + 1 # Próximo elemento
            else:
                next_state = STATE_WRITE_BACK # Terminou os 4 elementos
               
        # Escrita do Resultado Final (Acumulador)
        elif self.state == STATE_WRITE_BACK:
            self._set_outputs(we=1, acc_rst=0, mux_res_sel=1, addr_w=1, done=0) # WE=1, MUX_RES=ACC, ADDR_W=1 (Ex: Endereço 1)
            next_state = STATE_DONE
           
        # Conclusão da Operação
        elif self.state == STATE_DONE:
            self._set_outputs(we=0, acc_rst=0, mux_res_sel=0, done=1) # DONE = 1
            if start_in == 0: # Espera o sinal 'start' ser desativado
                next_state = STATE_IDLE


        # 2. Lógica Sequencial (Atualização do Estado)
        self.state = next_state
        return self.state, self.output_signals


    def _set_outputs(self, we, acc_rst, mux_res_sel, mux_sel=0, addr_w=0, done=0):
        """ Atualiza todos os sinais de controle de saída. """
        self.output_signals['we_ctrl'] = np.int8(we)
        self.output_signals['acc_rst'] = np.int8(acc_rst)
        self.output_signals['mux_2_1_res_sel'] = np.int8(mux_res_sel)
        self.output_signals['mux_4_1_sel'] = np.int8(mux_sel)
        self.output_signals['addr_w_ctrl'] = np.int8(addr_w)
        self.output_signals['done_out'] = np.int8(done)




def run_fsm_test(fsm_instance, test_id, start_in, expected_state, expected_outputs, test_desc):
    """Executa um ciclo de clock na FSM e verifica a transição e os sinais."""
   
    start_time = time.perf_counter()
    next_state, outputs = fsm_instance.update_state(start_in)
    end_time = time.perf_counter()
    duration = (end_time - start_time) * 1e6 # µs
   
    # 1. Verifica o Estado
    state_ok = (next_state == expected_state)
   
    # 2. Verifica os Sinais de Saída
    outputs_ok = True
    for key, expected_val in expected_outputs.items():
        if outputs[key] != expected_val:
            outputs_ok = False
            break
           
    verification = '✅ PASSOU' if state_ok and outputs_ok else '❌ FALHOU'
   
    print(f"\n--- Teste {test_id}: {test_desc} ---")
    print(f"  Estado Atual -> Próximo Estado: {fsm_instance.state} ({next_state}) | Esperado: {expected_state}")
    print(f"  Start_in: {start_in}")
    print(f"  Sinais Gerados: {outputs}")
    print(f"  Sinais Esperados: {expected_outputs}")
    print(f"  Tempo de Execução (Python): {duration:.3f} µs")
    print(f"  Verificação: {verification}")
   
    return next_state, outputs


# --- Execução dos Casos de Teste ---


print("## Testbench Máquina de Estados Finitos (FSM) - DOT Product ##")
fsm = FSM_Controller()
fsm.reset()
print(f"Estado Inicial: {fsm.state}")


# Simulação da execução completa do Dot Product (9 ciclos)


# 1. Caso de Borda: Inicial (IDLE -> LOAD_SETUP)
run_fsm_test(fsm, 1, 0, STATE_IDLE, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 0, 'done_out': 0},
             "IDLE (Start=0) - Permanece IDLE")


run_fsm_test(fsm, 2, 1, STATE_LOAD_SETUP, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 0, 'done_out': 0},
             "IDLE -> LOAD_SETUP (Start=1)")


# 2. Transição para Reset do Acumulador
run_fsm_test(fsm, 3, 1, STATE_LOAD_VECTORS, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 0, 'done_out': 0},
             "LOAD_SETUP -> LOAD_VECTORS")
             
run_fsm_test(fsm, 4, 1, STATE_ACCUMULATOR_RESET, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 0, 'done_out': 0},
             "LOAD_VECTORS -> ACCUMULATOR_RESET")


# 3. Caso Crítico: Reset do Acumulador (ACC_RST=1)
run_fsm_test(fsm, 5, 1, STATE_EXECUTE_0, {'we_ctrl': 0, 'acc_rst': 1, 'mux_4_1_sel': 0, 'done_out': 0},
             "ACC_RST -> EXECUTE_0 (Sinal ACC_RST ativado)")


# 4. Loop de Execução (Controle do MUX 4:1)
run_fsm_test(fsm, 6, 1, STATE_EXECUTE_1, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 0, 'done_out': 0},
             "EXECUTE_0 -> EXECUTE_1 (MUX_SEL=0)")
run_fsm_test(fsm, 7, 1, STATE_EXECUTE_2, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 1, 'done_out': 0},
             "EXECUTE_1 -> EXECUTE_2 (MUX_SEL=1)")
run_fsm_test(fsm, 8, 1, STATE_EXECUTE_3, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 2, 'done_out': 0},
             "EXECUTE_2 -> EXECUTE_3 (MUX_SEL=2)")


# 5. Caso de Borda do Loop (EXECUTE_3 -> WRITE_BACK)
run_fsm_test(fsm, 9, 1, STATE_WRITE_BACK, {'we_ctrl': 0, 'acc_rst': 0, 'mux_4_1_sel': 3, 'done_out': 0},
             "EXECUTE_3 -> WRITE_BACK (MUX_SEL=3)")


# 6. Caso Crítico: Escrita de Volta (WE=1, MUX_RES_SEL=ACC)
run_fsm_test(fsm, 10, 1, STATE_DONE, {'we_ctrl': 1, 'acc_rst': 0, 'mux_2_1_res_sel': 1, 'addr_w_ctrl': 1, 'done_out': 0},
             "WRITE_BACK -> DONE (WE ativado, MUX_RES=ACC)")


# 7. Caso Final: Conclusão (DONE=1) e Desativação do START
run_fsm_test(fsm, 11, 1, STATE_DONE, {'we_ctrl': 0, 'acc_rst': 0, 'mux_2_1_res_sel': 0, 'done_out': 1},
             "DONE (Start=1) - DONE ativado")


run_fsm_test(fsm, 12, 0, STATE_IDLE, {'we_ctrl': 0, 'acc_rst': 0, 'mux_2_1_res_sel': 0, 'done_out': 1},
             "DONE -> IDLE (Start=0)")
